<!-- ORM_Exam.ipynb -->
# Jup

In [1]:
# JupyterLab에서 django shell을 실행시키기 위한 설정
import os
import django

os.environ["DJANGO_SETTINGS_MODULE"] = "config.settings" # config/settings.py 모듈 지정
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

django.setup()

In [2]:
from polls.models import Question, Choice

c = Choice.objects.get(pk=1)
print(c)

1. 빨강색


In [4]:
# Model Manager -> Model클래스.objects => SQL을 실행하는 메소드들 제공.(DB작업 관리자자)
print(type(Choice.objects), type(Question.objects))

<class 'django.db.models.manager.Manager'> <class 'django.db.models.manager.Manager'>


In [5]:
# QuerySet 조회결과들을 담아주는 객체.
result = Choice.objects.all() # select * from tb(choice);
print(type(result))

<class 'django.db.models.query.QuerySet'>


In [7]:
# QuerySet: iterable
for r in result:
    print(r)

1. 빨강색
2. 파랑색
3. 빨강색
4. 검정색
5. 초록색
6. 검정색
7. 초록색
8. 호랑이
9. 사자
10. 개
11. 고양이
12. 뱀
13. 도마뱀
14. 하이에나
15. 파이썬
16. 자바
17. Javascript
18. C
19. 코끼리


In [10]:
# QuerySet: subscriptable 
result[1], len(result)

(<Choice: 2. 파랑색>, 19)

# 조회
- `Model.objects.all()` : 전체조회
- `Model.objects.filter(), exclude(), get()`
    - where절이 있는 조회

In [13]:
result_list = Question.objects.all()
print("조회결과 개수:", len(result_list))
print("실행된 SQL문조회 - QuerySet.query")
print(result_list.query)

조회결과 개수: 5
실행된 SQL문조회 - QuerySet.query
SELECT "polls_question"."id", "polls_question"."question_text", "polls_question"."pub_date" FROM "polls_question"


In [17]:
for result in result_list:
    #print(result)  #model.__str__() 
    # 개별 Field 조회(instance 변수)
    print(result.id, result.pk, end=",  ")  # id-primarykey field이름, pk: primary key
    print(result.question_text, result.pub_date)

1 1,  좋아하는 색은 무엇입니까? 2025-01-03 08:46:43.327472+00:00
2 2,  싫어하는 색은 무엇입니까? 2025-01-03 08:47:28.701171+00:00
3 3,  좋아하는 동물은 무엇인가요? 2025-01-06 00:46:18.116403+00:00
4 4,  싫어하는 동물은 무엇인가요? 2025-01-06 00:46:31.072882+00:00
5 5,  공부하고 싶은 프로그래밍 언어는 무엇입니까? 2025-01-06 00:46:55.941613+00:00


In [24]:
#  조회결과에서 개별 데이터 조회 - indexing
r = result_list[3]
print(r.pk, r.question_text, r.pub_date.strftime('%Y-%m-%d %H:%M:%S'))

4 싫어하는 동물은 무엇인가요? 2025-01-06 00:46:31


In [26]:
# QuerySet은 음수 index는 지원하지 않는다.
# result_list[-1]

In [34]:
# QuerySet.first() : 첫번째 조회값 . result_list[0]
# QuerySet.last()  :  마지막 조회값. result_list[-1]
result_list.first()
result_list.last()

<Question: 5. 공부하고 싶은 프로그래밍 언어는 무엇입니까?>

In [27]:
result_list[:3]  # slicing -반환타입-> list

[<Question: 1. 좋아하는 색은 무엇입니까?>,
 <Question: 2. 싫어하는 색은 무엇입니까?>,
 <Question: 3. 좋아하는 동물은 무엇인가요?>]

In [28]:
type(result_list[:3])

list

In [38]:
## 정렬 - sql: orderby 절
# queryset.order_by("field명") : asc, .order_by("-field명") : desc
## 기준 컬럼이 여러개일 경우 나열한다. 

# result_list = Question.objects.all()
# result_list = Question.objects.all().order_by("-pk") # id desc
result_list = Question.objects.all().order_by("question_text") # question_text asc
for result in result_list:
    print(result.pk, result.question_text, result.pub_date)

5 공부하고 싶은 프로그래밍 언어는 무엇입니까? 2025-01-06 00:46:55.941613+00:00
4 싫어하는 동물은 무엇인가요? 2025-01-06 00:46:31.072882+00:00
2 싫어하는 색은 무엇입니까? 2025-01-03 08:47:28.701171+00:00
3 좋아하는 동물은 무엇인가요? 2025-01-06 00:46:18.116403+00:00
1 좋아하는 색은 무엇입니까? 2025-01-03 08:46:43.327472+00:00


In [42]:
result_list = Choice.objects.all().order_by("-votes", "choice_text")
# SQL: orderby  votes desc, choice_text asc
for result in result_list:
    print(result.pk, result.choice_text, result.votes)

15 파이썬 200
8 호랑이 100
17 Javascript 51
10 개 50
16 자바 50
12 뱀 35
6 검정색 30
4 검정색 20
11 고양이 20
9 사자 20
14 하이에나 17
18 C 14
19 코끼리 11
7 초록색 10
13 도마뱀 5
5 초록색 3
1 빨강색 0
3 빨강색 0
2 파랑색 0


In [43]:
print(result_list.query)

SELECT "polls_choice"."id", "polls_choice"."choice_text", "polls_choice"."votes", "polls_choice"."question_id" FROM "polls_choice" ORDER BY "polls_choice"."votes" DESC, "polls_choice"."choice_text" ASC


## Where (조건)으로 조회
- `filter()`: 조회조건이 True행들을 조회 => QuerySet return
- `exclude()`: 조회조건이 False행들을 조회 => QuerySet return
- `get()` : 조회결과가 1개일 때만 사용(pk=값). 조회결과가 없거나 2개 이상이면 Exception발생. => Model return
- 조회조건 형식: `filed이름__연산자 = 값`

In [47]:
result = Question.objects.filter(pk=1)   # where pk = 1
result = Question.objects.get(pk=1)      # where pk = 1
result = Question.objects.exclude(pk=1)  # where not (pk = 1)
print(type(result))
result

<class 'django.db.models.query.QuerySet'>


<QuerySet [<Question: 2. 싫어하는 색은 무엇입니까?>, <Question: 3. 좋아하는 동물은 무엇인가요?>, <Question: 4. 싫어하는 동물은 무엇인가요?>, <Question: 5. 공부하고 싶은 프로그래밍 언어는 무엇입니까?>]>

In [57]:
### 비교 연산
result = Choice.objects.filter(pk__lt = 5) # where pk < 5
result = Choice.objects.filter(pk__lte = 5) # where pk <= 5
result = Choice.objects.filter(pk__gt = 15) # where pk > 15
result = Choice.objects.filter(pk__gte = 15) # where pk >= 15
result = Choice.objects.filter(choice_text = "빨강색")
for r in result:
    print(r)

1. 빨강색
3. 빨강색


In [58]:
print(result.query)

SELECT "polls_choice"."id", "polls_choice"."choice_text", "polls_choice"."votes", "polls_choice"."question_id" FROM "polls_choice" WHERE "polls_choice"."choice_text" = 빨강색


In [67]:
# 문자열 컬럼 부분일치 (like)
result = Question.objects.filter(question_text__contains = "색은")
# where question_text like '%색은%'

result = Question.objects.filter(question_text__startswith = "좋아하는")
# where question_text like '좋아하는%'

result = Question.objects.filter(question_text__endswith = "무엇입니까?")
# where question_text like '%무엇입니까?'

result = Question.objects.exclude(question_text__endswith = "무엇입니까?")

print(result.query)
for r in result:
    print(r)

SELECT "polls_question"."id", "polls_question"."question_text", "polls_question"."pub_date" FROM "polls_question" WHERE NOT ("polls_question"."question_text" LIKE %무엇입니까? ESCAPE '\')
3. 좋아하는 동물은 무엇인가요?
4. 싫어하는 동물은 무엇인가요?


In [66]:
## where 컬럼 in [v1, v2, v3, ..]
Question.objects.filter(pk__in = [1, 3, 4])
Question.objects.exclude(pk__in = [1, 3, 4])  # not

<QuerySet [<Question: 2. 싫어하는 색은 무엇입니까?>, <Question: 5. 공부하고 싶은 프로그래밍 언어는 무엇입니까?>]>

In [68]:
## where 컬럼 between A and B 
Choice.objects.filter(pk__range = [2, 6]) # pk between 2 and 6

<QuerySet [<Choice: 2. 파랑색>, <Choice: 3. 빨강색>, <Choice: 4. 검정색>, <Choice: 5. 초록색>, <Choice: 6. 검정색>]>

#### where 절의 and, or
- `AND`: 조건을 나열한다.
- `OR` : 각 조건을 Q()함수에 넣고 `|` 로 연결한다.

In [70]:
result = Question.objects.filter(
    question_text__endswith = "무엇입니까?",
    pk__lt = 3
)

print(result.query)
for r in result:
    print(r)

SELECT "polls_question"."id", "polls_question"."question_text", "polls_question"."pub_date" FROM "polls_question" WHERE ("polls_question"."id" < 3 AND "polls_question"."question_text" LIKE %무엇입니까? ESCAPE '\')
1. 좋아하는 색은 무엇입니까?
2. 싫어하는 색은 무엇입니까?


In [73]:
# or
from django.db.models import Q

# OR : 조건을 Q에 넣고 `|` 연결
result = Question.objects.filter(
    Q(question_text__endswith = "무엇입니까?") | Q(pk__gte = 3)
)

# ~Q(조건) : not 조건
##  AND 나 OR 로 조회조건을 합칠때 개별 조건에 NOT을 붙이려면 Q()를 이용.
result = Question.objects.filter(
    ~Q(question_text__endswith = "무엇입니까?") | Q(pk__gte = 3)
)

print(result.query)
for r in result:
    print(r)


SELECT "polls_question"."id", "polls_question"."question_text", "polls_question"."pub_date" FROM "polls_question" WHERE (NOT ("polls_question"."question_text" LIKE %무엇입니까? ESCAPE '\') OR "polls_question"."id" >= 3)
3. 좋아하는 동물은 무엇인가요?
4. 싫어하는 동물은 무엇인가요?
5. 공부하고 싶은 프로그래밍 언어는 무엇입니까?


### 조회 컬럼을 지정
- `values(컬럼명, ..)`
    - 개별(행) 조회 결과를 dictionary로 반환.

In [82]:
result = Question.objects.all().values('pk', 'question_text')
result = Question.objects.filter(pk__lt = 3).values('pk', 'question_text')

result

<QuerySet [{'pk': 1, 'question_text': '좋아하는 색은 무엇입니까?'}, {'pk': 2, 'question_text': '싫어하는 색은 무엇입니까?'}]>

In [83]:
print(result.query)

SELECT "polls_question"."id", "polls_question"."question_text" FROM "polls_question" WHERE "polls_question"."id" < 3


In [88]:
try:
    a = Question.objects.get(pk=20)
    print(a)
except:
    print('조회결과가 없습니다.')

조회결과가 없습니다.
